In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
from src.SWATPollution import SWATPollution
from pathlib import Path
from src.pollution_utils import generate_pollution_observations, observacions_from_conca
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


In [40]:
contaminant = 'Ciprofloxacina'
conca = 'besos'

In [41]:
cwd = Path('C:\\Users\\jsalo\\Desktop\\ICRA\\traca_contaminacio\\traca_contaminacio')
txtinout_folder = cwd/'data'/'txtinouts'/'tmp'/contaminant/conca
channels_geom_path = cwd/'data'/'rivs1'/'canals_tot_ci.shp'
lod_path = cwd / 'data' / 'lod.xlsx'

In [42]:
observacions = generate_pollution_observations(contaminant)
df = observacions_from_conca(channels_geom_path, observacions, conca)


c:\Users\jsalo\Desktop\ICRA\traca_contaminacio\traca_contaminacio\src\pollution_utils.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  #Convert concentration of observations to mg/L


In [43]:
first_observation = df.year.min()
year_end = 2022
year_start = max(first_observation-3, 2000) #3 years warm-up
warmup = max(1, first_observation - year_start)

In [44]:
lod_df = pd.read_excel(lod_path, index_col=0)
lod = lod_df.loc[contaminant, 'LOD (mg/L)']


In [45]:

"""
swatpy = SWATPollution( 
        conca, 
        contaminant, 
        txtinout_folder = cwd/'data'/'txtinouts'/f"TxtInOut_{conca}",
        channels_geom_path = cwd/'data'/'rivs1'/'canals_tot_ci.shp',
        tmp_path = cwd/'data'/'txtinouts'/'tmp'/conca,
        run = True,
        compound_features = {},
        show_output = True,
        copy_txtinout=True,
        overwrite_txtinout=True
    )


"""
swatpy = SWATPollution( 
        conca, 
        contaminant, 
        txtinout_folder = txtinout_folder,
        channels_geom_path = channels_geom_path,
        run = False,
        observacions = observacions.copy(),
        lod = lod,
        year_start = year_start,
        year_end = year_end,
        warmup = warmup
)


In [46]:
gis_id = 81

In [47]:
predictions = swatpy.get_df()  
observations = swatpy.gdf_observacions.copy()

predictions_channel = predictions[predictions['gis_id'] == gis_id][['mg_l', 'Date']]
observations_channel = observations[observations['gis_id'] == gis_id]

In [48]:
predictions_channel

,mg_l,Date
116880,0.000009,2019-01-01
116881,0.000029,2019-01-02
116882,0.000028,2019-01-03
116883,0.000029,2019-01-04
116884,0.000027,2019-01-05
...,...,...
118336,0.000009,2022-12-27
118337,0.000039,2022-12-28
118338,0.000027,2022-12-29
118339,0.000038,2022-12-30


In [49]:
observations

,fecha,estacion,cod_estaci,utm_x,utm_y,variable,unidad_med,valor_alfa,valor,gis_id,...,day,mon,yr,pollutant,mg_l,flo_out,tot_out_mg,error,geometry,fecha_str
0,2019-03-13,QLSup - Riu Ripoll des de l'EDAR de Sabadell f...,110025000,432000,4593213,Ciprofloxacina,ng/L,59,0.000059,125,...,13,3.0,2019.0,Ciprofloxacina,0.000121,1.8360,19204.850,0.000062,POINT (2.18542 41.48765),2019-03-13
1,2019-03-13,QLSup - Riu Sec (Bess),110026000,429192,4593550,Ciprofloxacina,ng/L,121,0.000121,72,...,13,3.0,2019.0,Ciprofloxacina,0.000174,0.7009,10525.658,0.000053,POINT (2.15175 41.49044),2019-03-13
2,2019-02-25,QLSup - Riu Congost des de l'EDAR d'Aiguafreda...,110004000,439609,4619390,Ciprofloxacina,ng/L,69,0.000069,127,...,25,2.0,2019.0,Ciprofloxacina,0.000093,0.1222,985.251,0.000024,POINT (2.27393 41.72402),2019-02-25
3,2019-03-13,QLSup - Riu Ripoll des de l' EDAR de Castellar...,110024000,425278,4602157,Ciprofloxacina,ng/L,43,0.000043,81,...,13,3.0,2019.0,Ciprofloxacina,0.000031,0.3020,820.884,0.000012,POINT (2.10380 41.56760),2019-03-13
4,2019-03-13,QLSup - El Bess des de la confluncia del Ripol...,110030000,433240,4589118,Ciprofloxacina,ng/L,208,0.000208,78,...,13,3.0,2019.0,Ciprofloxacina,0.000188,6.0350,98242.340,0.000020,POINT (2.20073 41.45087),2019-03-13
5,2019-02-25,QLSup - Riu Congost des de la confluncia de la...,110006000,437707,4600165,Ciprofloxacina,ng/L,244,0.000244,19,...,25,2.0,2019.0,Ciprofloxacina,0.000144,0.7578,9456.792,0.000100,POINT (2.25306 41.55072),2019-02-25
6,2019-02-25,QLSup - Riu Congost des de l'EDAR de la Garrig...,110005000,441159,4612020,Ciprofloxacina,ng/L,115,0.000115,24,...,25,2.0,2019.0,Ciprofloxacina,0.000118,0.2233,2275.731,0.000003,POINT (2.29329 41.65776),2019-02-25


In [54]:
observations_channel

,fecha,estacion,cod_estaci,utm_x,utm_y,variable,unidad_med,valor_alfa,valor,gis_id,...,day,mon,yr,pollutant,mg_l,flo_out,tot_out_mg,error,geometry,fecha_str
3,2019-03-13,QLSup - Riu Ripoll des de l' EDAR de Castellar...,110024000,425278,4602157,Ciprofloxacina,ng/L,43,0.000043,81,...,13,3.0,2019.0,Ciprofloxacina,0.000031,0.302,820.884,0.000012,POINT (2.10380 41.56760),2019-03-13


In [55]:
#rmse
swatpy.get_error()  

-0.5000403980244833

In [65]:
predictions_channel

,mg_l,Date
116880,0.000009,2019-01-01
116881,0.000029,2019-01-02
116882,0.000028,2019-01-03
116883,0.000029,2019-01-04
116884,0.000027,2019-01-05
...,...,...
118336,0.000009,2022-12-27
118337,0.000039,2022-12-28
118338,0.000027,2022-12-29
118339,0.000038,2022-12-30


In [83]:
fig = px.line(predictions_channel, x='Date', y="mg_l", )



fig.add_trace(
    px.scatter(observations_channel, x='fecha', y="valor").update_traces(marker=dict(color='orange'), mode='markers').data[0]
)

fig.update_traces(line=dict(color='black'))


rmse = swatpy.rmse

upper_bound = predictions_channel['mg_l'] + 2*rmse
lower_bound = predictions_channel['mg_l'] - 2*rmse
lower_bound = lower_bound.apply(lambda x: max(0, x))

fig.add_traces([go.Scatter(x = predictions_channel.Date, y = upper_bound,
                           mode = 'lines', line_color = 'rgba(0,0,0,0)',
                           showlegend = False),
                go.Scatter(x = predictions_channel.Date, y = lower_bound,
                           mode = 'lines', line_color = 'rgba(0,0,0,0)',
                           name = '95% confidence interval',
                           fill='tonexty', fillcolor = 'rgba(128, 128, 128, 0.2)')])


fig.update_traces(marker=dict(size=9,
                                      color = 'orange',
                                      line=dict(width=1)),
                            selector=dict(mode='markers'))


fig.update_layout(
            showlegend=False,
            plot_bgcolor="white",
            margin=dict(t=10,l=10,b=10,r=10)
        )
        
fig.update_layout(
            plot_bgcolor='white'
        )
fig.update_xaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey'
        )
fig.update_yaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey'
        )



fig.show()